In [45]:

import torch
import os
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
import matplotlib
matplotlib.use('Agg')                     #This is for using AGG backend in order to prevent failure of memory while creating the images
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda':
    print(torch.cuda.get_device_name())
class FCN(nn.Module):
    "Defines a fully-connected network in PyTorch"
    def __init__(self,N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = nn.Tanh
        self.fcs = nn.Sequential(*[
                        nn.Linear(N_INPUT, N_HIDDEN),
                        activation()])
        self.fch = nn.Sequential(*[
                        nn.Sequential(*[
                            nn.Linear(N_HIDDEN, N_HIDDEN),
                            activation()]) for _ in range(N_LAYERS-1)])
        self.fce = nn.Linear(N_HIDDEN, N_OUTPUT)
    def forward(self, x):
        x = self.fcs(x)
        x = self.fch(x)
        x = self.fce(x)
        return x
def materials(x,y):
    d_blanket=torch.add(torch.zeros_like(x),2.094999864)
    d_core = torch.add(torch.zeros_like(x),2.200801092)
    s_blanket= torch.add(torch.zeros_like(x),0.00214231)
    s_core= torch.add(torch.zeros_like(x),0.01048083)
    sigma_a_blanket=torch.add(torch.zeros_like(x),0.064256)
    sigma_a_core =torch.add(torch.zeros_like(x),0.062158)
    sigma_s_blanket= torch.add(torch.zeros_like(x),0.094853)
    sigma_s_core= torch.add(torch.zeros_like(x),0.089302)
    c1= ((x >= 10) & (x <= 30)) & ((y >= 10) & (y <= 30))
    c2= (x >= 70) & (x <= 90) & (y >= 70) & (y <= 90)
    region= c1|c2
    d=torch.where(region,d_core,d_blanket)
    s=torch.where(region,s_core,s_blanket)
    sigma_a=torch.where(region,sigma_a_core,sigma_a_blanket)
    sigma_s=torch.where(region,sigma_s_core,sigma_a_blanket)

    return d, s, sigma_a,sigma_s

N_b=1000
N_f=3000
#Test_point=1000

X_lb = np.array([0.0, 0.0]) + np.array([100.0, 0.0]) * lhs(2,  int(N_b))
X_ub = np.array([0.0, 100.0]) + np.array([100.0, 0.0]) * lhs(2,  int(N_b))
Y_lb = np.array([0.0, 0.0]) + np.array([0.0, 100.0]) * lhs(2,  int(N_b))
Y_rb = np.array([100.0, 0.0]) + np.array([0.0, 100.0]) * lhs(2,  int(N_b))
X_f  = np.array([0.0,0.0])+ np.array([100.0,100.0])*lhs(2,int(N_f))
X_test = np.array([0.0,0.0])+ np.array([100.0,100.0])*lhs(2,int(Test_point))


torch.manual_seed(123)

pinn= FCN(2,1,40,8).to(device)
lower=torch.tensor(X_lb, dtype=torch.float32, requires_grad=True).to(device)
upper=torch.tensor(X_ub, dtype=torch.float32, requires_grad=True).to(device)
left = torch.tensor(Y_lb, dtype=torch.float32, requires_grad=True).to(device)
right=torch.tensor(Y_rb, dtype=torch.float32, requires_grad=True).to(device)
train=torch.tensor(X_f, dtype=torch.float32, requires_grad=True).to(device)
d,s,sig_a,sig_s=materials(train[:,0],train[:,1])
d=d.to(device)
s=s.to(device)
sig_a=sig_a.to(device)
sig_s=sig_s.to(device)
#test=torch.tensor(X_test,dtype=torch.float32)
x=np.linspace(0,100,100)
y=np.linspace(0,100,100)
X,Y= np.meshgrid(x,y)
test= torch.tensor(np.vstack([X.flatten(), Y.flatten()]).T, dtype=torch.float32).cpu()
test=test.to(device)
optimiser= torch.optim.Adam(pinn.parameters(),lr=1e-3)
scheduler = optim.lr_scheduler.StepLR(optimiser, step_size=2000, gamma=0.95)
# Early Stopping and Training Loop
best_loss = float('inf')
early_stop_counter = 0
early_stopping_patience = 10

plot_dir = '60_8_1000_3000_1000_test_45000steps_d_age'
os.makedirs(plot_dir, exist_ok=True)

for i in range(45001):
    optimiser.zero_grad()

    u=pinn(lower)
    dudy=torch.autograd.grad(u,lower,torch.ones_like(u),create_graph=True)[0]
    loss_low=torch.mean((.5*u-(2.094999864*dudy[:,[1]]))**2)
    u=pinn(upper)
    dudy=torch.autograd.grad(u,upper,torch.ones_like(u),create_graph=True)[0]
    loss_ub=torch.mean((.5*u+(2.094999864*dudy[:,[1]]))**2)
    u=pinn(left)
    dudx=torch.autograd.grad(u,left,torch.ones_like(u),create_graph=True)[0]
    loss_lb=torch.mean((.5*u-(2.094999864*dudx[:,[0]]))**2)
    u=pinn(right)
    dudx=torch.autograd.grad(u,right,torch.ones_like(u),create_graph=True)[0]
    loss_rb=torch.mean((.5*u+(2.094999864*dudx[:,[0]]))**2)
    #define Physics loss
    # Compute the gradient (first derivatives) of u with respect to x and y
    u=pinn(train)
    grad_u= torch.autograd.grad(u,train,torch.ones_like(u),create_graph=True)[0]
    grad_u_x = d*grad_u[:,0]
    grad_u_y = d*grad_u[:,1]

# Now we need to compute the second derivatives
    grad_u_xx=torch.autograd.grad(grad_u_x, train, grad_outputs=torch.ones_like(grad_u_x), create_graph=True)[0]
    grad_u_yy=torch.autograd.grad(grad_u_y, train, grad_outputs=torch.ones_like(grad_u_y), create_graph=True)[0]
    d2u_dx2 =grad_u_xx[:,0]   # Second derivative with respect to x
    d2u_dy2 = grad_u_yy[:,1]  # Second derivative with respect to y
    loss_phy=torch.mean((-s-(d2u_dx2+d2u_dy2)+(sig_a*u))**2)
    loss=loss_low+loss_ub+loss_lb+loss_rb+loss_phy
    loss.backward()
    optimiser.step()
    scheduler.step()

    if i %1000==0:
        #print(test.shape)
        phi=pinn(test).detach().cpu()
        phid=phi.view(X.shape)
        print(loss_low,loss_ub, loss_lb , loss_rb,loss_phy,loss)
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        #phi=phi.reshape(len(x),len(x))
        surf = ax.plot_surface(X,Y,phid,cmap='viridis')
        # Add labels and title
        ax.set_xlabel('X-axis')
        ax.set_ylabel('Y-axis')
        ax.set_zlabel('Flux')
        ax.set_title(f'Training Step{i}')
    

        # Add colorbar
        fig.colorbar(surf, ax=ax, label='Flux')
        fig.savefig(os.path.join(plot_dir, f'plot_epoch_{i}.png'))
        

        # Show the plot
        #plt.show()
        plt.figure(figsize=(10, 8))
        plt.contourf(X,Y,phid, levels=100)
        plt.colorbar()
        #plt.title('Neutron Flux')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title(f'Training Step {i}')
        plt.savefig(os.path.join(plot_dir, f'plot_epoch_heat{i}.png'))
        #plt.show()
        plt.close()
        if loss.item() < best_loss:
            best_loss = loss.item()
            early_stop_counter = 0
            # Saving the model state
            torch.save(pinn, os.path.join(plot_dir, f'pinn_model_step_{i}.pth'))
        else:
            early_stop_counter += 1
            if early_stop_counter >= early_stopping_patience:
                print(f"Stopping early at step {i}")
                continue

torch.save(pinn, os.path.join(plot_dir, 'final_pinn_model_best.pth'))

    


cuda
tensor(0.0002, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.0004, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.0003, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.0002, device='cuda:0', grad_fn=<MeanBackward0>) tensor(5.6556e-06, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.0011, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5.2823e-10, device='cuda:0', grad_fn=<MeanBackward0>) tensor(4.8193e-08, device='cuda:0', grad_fn=<MeanBackward0>) tensor(7.5117e-10, device='cuda:0', grad_fn=<MeanBackward0>) tensor(3.9973e-08, device='cuda:0', grad_fn=<MeanBackward0>) tensor(1.2429e-05, device='cuda:0', grad_fn=<MeanBackward0>) tensor(1.2518e-05, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.6383e-09, device='cuda:0', grad_fn=<MeanBackward0>) tensor(4.9918e-08, device='cuda:0', grad_fn=<MeanBackward0>) tensor(1.5593e-09, device='cuda:0', grad_fn=<MeanBackward0>) tensor(4.2466e-08, device='cuda:0', grad_fn=<MeanBackward0>) tensor(1.1708e-05, device='cuda:0', grad_

In [7]:

torch.save(pinn, os.path.join(plot_dir, '/final_pinn_model_full.pth'))

# Printing And Plotting

In [1]:
model=torch.load('./40_8_1000_5000_1000_test/final_pinn_model_full.pth')

NameError: name 'torch' is not defined

In [17]:
import matplotlib

phi=model(test).detach().cpu()
phid=phi.view(X.shape)
print(loss_low,loss_ub, loss_lb , loss_rb,loss_phy,loss)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#phi=phi.reshape(len(x),len(x))
surf = ax.plot_surface(X,Y,phid,cmap='viridis')
# Add labels and title
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Flux')
ax.set_title(f'Training Step{i}')


# Add colorbar
fig.colorbar(surf, ax=ax, label='Flux')
fig.savefig(os.path.join(plot_dir, f'plot_epoch_{i}.png'))


# Show the plot
plt.show()
print(phid)

tensor(5.6396e-08, device='cuda:0', grad_fn=<MeanBackward0>) tensor(1.4505e-08, device='cuda:0', grad_fn=<MeanBackward0>) tensor(4.2022e-07, device='cuda:0', grad_fn=<MeanBackward0>) tensor(5.8596e-08, device='cuda:0', grad_fn=<MeanBackward0>) tensor(8.4593e-06, device='cuda:0', grad_fn=<MeanBackward0>) tensor(9.0090e-06, device='cuda:0', grad_fn=<AddBackward0>)
tensor([[-5.2832e-04, -5.7699e-04, -5.6263e-04,  ..., -7.1064e-05,
         -1.2806e-04, -1.8538e-04],
        [-3.0377e-04, -3.0778e-04, -3.2734e-04,  ..., -3.6456e-05,
         -1.0172e-04, -1.6698e-04],
        [-8.2240e-05, -2.7530e-06,  4.5642e-05,  ...,  6.6876e-05,
         -1.1008e-05, -8.8502e-05],
        ...,
        [ 3.6534e-04,  5.3443e-04,  6.8678e-04,  ...,  1.4722e-03,
          1.4392e-03,  1.4111e-03],
        [ 3.2083e-04,  4.8183e-04,  6.2523e-04,  ...,  1.3724e-03,
          1.3362e-03,  1.3089e-03],
        [ 2.7863e-04,  4.3214e-04,  5.6734e-04,  ...,  1.2742e-03,
          1.2405e-03,  1.2096e-03]])


C:\Users\88018\AppData\Local\Temp\ipykernel_49524\3874817862.py:23: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [3]:

import torch
import os
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
import matplotlib
matplotlib.use('Agg')                     #This is for using AGG backend in order to prevent failure of memory while creating the images
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda':
    print(torch.cuda.get_device_name())
class FCN(nn.Module):
    "Defines a fully-connected network in PyTorch"
    def __init__(self,N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = nn.Tanh
        self.fcs = nn.Sequential(*[
                        nn.Linear(N_INPUT, N_HIDDEN),
                        activation()])
        self.fch = nn.Sequential(*[
                        nn.Sequential(*[
                            nn.Linear(N_HIDDEN, N_HIDDEN),
                            activation()]) for _ in range(N_LAYERS-1)])
        self.fce = nn.Linear(N_HIDDEN, N_OUTPUT)
    def forward(self, x):
        x = self.fcs(x)
        x = self.fch(x)
        x = self.fce(x)
        return x

cuda


In [43]:
import pandas as pd
import torch

# Load the trained PINN model (make sure to replace this with the correct path to your saved model)
pinn_model_path = './40_8_1000_5000_1000_test_35000steps_d_age/final_pinn_model_best.pth'
pinn_model = torch.load(pinn_model_path, map_location=device)

# Load the FEM results from the CSV file
fem_results = pd.read_csv('./LCRM_sol.csv')

num_points = len(fem_results)
x = np.linspace(0, 100, num_points)
y = np.linspace(0, 100, num_points)
X, Y = np.meshgrid(x, y)

# Flatten the meshgrid coordinates and FEM results for comparison
X_flat = X.flatten()
Y_flat = Y.flatten()
U_fem_flat = fem_results.values.flatten()
X_fem_tensor = torch.tensor(np.vstack([X_flat, Y_flat]).T, dtype=torch.float32).to(device)
# # Convert to tensor
# X_fem_tensor = torch.tensor(X_fem, dtype=torch.float32).to(device)
# Generate PINN predictions
pinn_model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    U_pinn = pinn_model(X_fem_tensor).cpu().numpy().flatten()

# Calculate MSE
mse_error = ((U_pinn - U_fem_flat) ** 2).mean()
print(f'Mean Squared Error: {mse_error}')

# Calculate the relative error
relative_error = np.sqrt(mse_error) / np.abs(U_fem_flat).mean()
print(f'Relative Error: {relative_error}')
# Calculate the relative percentage error
relative_percentage_error = (np.linalg.norm(U_pinn - U_fem_flat) / np.linalg.norm(U_fem_flat)) * 100

print(f'Relative Percentage Error: {relative_percentage_error}%')

Mean Squared Error: 0.000218194610140953
Relative Error: 0.37470449966484715
Relative Percentage Error: 32.40195869938062%


In [41]:
print(pinn_model(X_fem_tensor).detach().cpu().numpy())
point_df=pd.DataFrame(pinn_model(X_fem_tensor).detach().cpu().numpy())
point_df.to_csv('model_pred_index.csv', index=True)

[[0.00524517]
 [0.00604459]
 [0.00664264]
 ...
 [0.00405727]
 [0.00350349]
 [0.00286959]]


In [31]:
import pandas as pd
import torch

# Load the trained PINN model (make sure to replace this with the correct path to your saved model)
pinn_model_path = './40_8_1000_5000_1000_test_35000steps/final_pinn_model_best.pth'
pinn_model = torch.load(pinn_model_path, map_location=device)

# Load the FEM results from the CSV file
fem_results = pd.read_csv('./LCRM_sol.csv')

num_points = len(fem_results)
x = np.linspace(0, 100, num_points)
y = np.linspace(0, 100, num_points)
X, Y = np.meshgrid(x, y)

# Flatten the meshgrid coordinates and FEM results for comparison
X_flat = X.flatten()
Y_flat = Y.flatten()
U_fem_flat = fem_results.values.flatten()
X_fem_tensor = torch.tensor(np.vstack([X_flat, Y_flat]).T, dtype=torch.float32).to(device)
# # Convert to tensor
# X_fem_tensor = torch.tensor(X_fem, dtype=torch.float32).to(device)
# Generate PINN predictions
pinn_model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    U_pinn = pinn_model(X_fem_tensor).cpu().numpy().flatten()

# Calculate MSE
mse_error = ((U_pinn - U_fem_flat) ** 2).mean()
print(f'Mean Squared Error: {mse_error}')

# Calculate the relative error
relative_error = np.sqrt(mse_error) / np.abs(U_fem_flat).mean()
print(f'Relative Error: {relative_error}')
# Calculate the relative percentage error
relative_percentage_error = (np.linalg.norm(U_pinn - U_fem_flat) / np.linalg.norm(U_fem_flat)) * 100

print(f'Relative Percentage Error: {relative_percentage_error}%')

Mean Squared Error: 0.0002259812871847695
Relative Error: 0.38133190075884765
Relative Percentage Error: 32.97505236845614%


In [14]:
import pandas as pd
import torch
import numpy as np
import os

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the FEM results from the CSV file
fem_results_path = './LCRM_sol.csv'
fem_results = pd.read_csv(fem_results_path)

# Flatten the FEM results for comparison
U_fem_flat = fem_results.values.flatten()

# Generate the meshgrid for the domain
num_points = int(np.sqrt(len(U_fem_flat)))  # Assuming a square domain
x = np.linspace(0, 100, num_points)
y = np.linspace(0, 100, num_points)
X, Y = np.meshgrid(x, y)
X_flat = X.flatten()
Y_flat = Y.flatten()
X_fem_tensor = torch.tensor(np.vstack([X_flat, Y_flat]).T, dtype=torch.float32).to(device)

# Directory where the PINN models are saved
pinn_models_dir = './40_8_1000_5000_1000_test_70000steps/stages/'

# Initialize a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Step', 'MSE', 'Relative_Error', 'Relative_Percentage_Error'])

# Loop through the saved models and calculate errors
for filename in sorted(os.listdir(pinn_models_dir)):
    if filename.endswith('.pth'):
        step = int(filename.split('_')[-1].split('.')[0])  # Extract step number from filename
        pinn_model_path = os.path.join(pinn_models_dir, filename)
        pinn_model = torch.load(pinn_model_path, map_location=device)
        pinn_model.eval()  # Set the model to evaluation mode

        with torch.no_grad():
            U_pinn = pinn_model(X_fem_tensor).cpu().numpy().flatten()

        # Calculate MSE
        mse_error = ((U_pinn - U_fem_flat) ** 2).mean()

        # Calculate the relative error
        relative_error = np.sqrt(mse_error) / np.abs(U_fem_flat).mean()

        # Calculate the relative percentage error
        relative_percentage_error = (np.linalg.norm(U_pinn - U_fem_flat) / np.linalg.norm(U_fem_flat)) * 100

        # Append the errors to the DataFrame
        error_df = error_df.append({
            'Step': step,
            'MSE': mse_error,
            'Relative_Error': relative_error,
            'Relative_Percentage_Error': relative_percentage_error
        }, ignore_index=True)

# Save the errors DataFrame to a CSV file
error_df.to_csv('pinn_errors.csv', index=False)
print("Errors saved to pinn_errors.csv")


ValueError: invalid literal for int() with base 10: 'best'

In [ ]:
import pandas as pd
import torch
import numpy as np
import os

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the FEM results from the CSV file
fem_results_path = './LCRM_sol.csv'
fem_results = pd.read_csv(fem_results_path)

# Flatten the FEM results for comparison
U_fem_flat = fem_results.values.flatten()

# Generate the meshgrid for the domain
num_points = int(np.sqrt(len(U_fem_flat)))  # Assuming a square domain
x = np.linspace(0, 100, num_points)
y = np.linspace(0, 100, num_points)
X, Y = np.meshgrid(x, y)
X_flat = X.flatten()
Y_flat = Y.flatten()
X_fem_tensor = torch.tensor(np.vstack([X_flat, Y_flat]).T, dtype=torch.float32).to(device)

# Directory where the PINN models are saved
pinn_models_dir = './40_8_1000_5000_1000_test_70000steps/stages/'

# Initialize a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Step', 'MSE', 'Relative_Error', 'Relative_Percentage_Error'])

# Loop through the saved models and calculate errors
for filename in sorted(os.listdir(pinn_models_dir)):
    if filename.endswith('.pth'):
        try:
            # Extract step number from filename, assuming it's formatted like "pinn_model_step_{step}.pth"
            step = int(filename.split('_')[-1].split('.')[0])
        except ValueError:
            # Skip files that do not have a step number in their name
            print(f"Skipping file with unexpected name format: {filename}")
            continue

        pinn_model_path = os.path.join(pinn_models_dir, filename)
        pinn_model = torch.load(pinn_model_path, map_location=device)
        pinn_model.eval()  # Set the model to evaluation mode

        with torch.no_grad():
            U_pinn = pinn_model(X_fem_tensor).cpu().numpy().flatten()

        # Calculate MSE
        mse_error = ((U_pinn - U_fem_flat) ** 2).mean()

        # Calculate the relative error
        relative_error = np.sqrt(mse_error) / np.abs(U_fem_flat).mean()

        # Calculate the relative percentage error
        relative_percentage_error = (np.linalg.norm(U_pinn - U_fem_flat) / np.linalg.norm(U_fem_flat)) * 100

        # Append the errors to the DataFrame
        error_df = error_df.append({
            'Step': step,
            'MSE': mse_error,
            'Relative_Error': relative_error,
            'Relative_Percentage_Error': relative_percentage_error
        }, ignore_index=True)

# Save the errors DataFrame to a CSV file
error_df.to_csv('pinn_errors.csv', index=False)
print("Errors saved to pinn_errors.csv")


In [25]:
import pandas as pd
import torch
import numpy as np
import os

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the FEM results from the CSV file
fem_results_path = './LCRM_sol.csv'
fem_results = pd.read_csv(fem_results_path)

# Flatten the FEM results for comparison
U_fem_flat = fem_results.values.flatten()

# Generate the meshgrid for the domain
num_points = int(np.sqrt(len(U_fem_flat)))  # Assuming a square domain
x = np.linspace(0, 100, num_points)
y = np.linspace(0, 100, num_points)
X, Y = np.meshgrid(x, y)
X_flat = X.flatten()
Y_flat = Y.flatten()
X_fem_tensor = torch.tensor(np.vstack([X_flat, Y_flat]).T, dtype=torch.float32).to(device)

# Directory where the PINN models are saved
pinn_models_dir = './40_8_500_1500_1000_test_35000steps/'

# Initialize a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Step', 'MSE', 'Relative_Error', 'Relative_Percentage_Error'])

# Loop through the saved models and calculate errors
for filename in sorted(os.listdir(pinn_models_dir)):
    if filename.endswith('.pth'):
        try:
            # Extract step number from filename, assuming it's formatted like "pinn_model_step_{step}.pth"
            step = int(filename.split('_')[-1].split('.')[0])
        except ValueError:
            # Skip files that do not have a step number in their name
            print(f"Skipping file with unexpected name format: {filename}")
            continue

        pinn_model_path = os.path.join(pinn_models_dir, filename)
        pinn_model = torch.load(pinn_model_path, map_location=device)
        pinn_model.eval()  # Set the model to evaluation mode

        with torch.no_grad():
            U_pinn = pinn_model(X_fem_tensor).cpu().numpy().flatten()

        # Calculate MSE
        mse_error = ((U_pinn - U_fem_flat) ** 2).mean()

        # Calculate the relative error
        relative_error = np.sqrt(mse_error) / np.abs(U_fem_flat).mean()

        # Calculate the relative percentage error
        relative_percentage_error = (np.linalg.norm(U_pinn - U_fem_flat) / np.linalg.norm(U_fem_flat)) * 100

        # Append the errors to the DataFrame
        error_df = error_df.append({
            'Step': step,
            'MSE': mse_error,
            'Relative_Error': relative_error,
            'Relative_Percentage_Error': relative_percentage_error
        }, ignore_index=True)

# Save the errors DataFrame to a CSV file
error_df.to_csv(f'{pinn_models_dir}erros.csv', index=False)
print("Errors saved to pinn_errors.csv")


Skipping file with unexpected name format: final_pinn_model_best.pth


C:\Users\88018\AppData\Local\Temp\ipykernel_13936\1245683626.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\88018\AppData\Local\Temp\ipykernel_13936\1245683626.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\88018\AppData\Local\Temp\ipykernel_13936\1245683626.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\88018\AppData\Local\Temp\ipykernel_13936\1245683626.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\88018\AppData\Local\Temp\ipykernel_13936\1245683626.py:59: Futu

Errors saved to pinn_errors.csv
